# TSP
### 都市の位置を乱数で生成

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def gen_random_tsp(ncity: int):
    # 座標
    locations = np.random.uniform(size=(ncity, 2))

    # 距離行列
    all_diffs = np.expand_dims(locations, axis=1) - np.expand_dims(locations, axis=0)
    distances = np.sqrt(np.sum(all_diffs ** 2, axis=-1))

    return locations, distances

def show_plot(locs: np.ndarray):
    plt.figure(figsize=(7, 7))
    plt.xlabel("x")
    plt.ylabel("y")
    plt.scatter(*zip(*locs))
    plt.show()

### 都市位置の生成
<span style="color: red; ">問題サイズはここで決める</span>

In [ ]:
# 都市数
ncity = 30
locations, distances = gen_random_tsp(ncity)

### 都市位置のプロット

In [ ]:
show_plot(locations)

# <span style="color: blue; ">QUBO定式化</span>
## 決定変数生成
２値変数 $q_{n,i}$ : $n$ 順番のインデックス、$i$ 都市のインデックス

In [ ]:
from amplify import BinarySymbolGenerator

gen = BinarySymbolGenerator()
# ２値変数 q_{i,j} : 都市 i から都市 j への経路を選択するとき 1
q = gen.array(ncity, ncity)

### <span style="color: red; ">回転対称性の除去</span>
0番目の都市から開始する

In [ ]:
q[0,0] = 1
for i in range(1,ncity):
    q[0,i] = 0
    q[i,0] = 0

In [ ]:
q

### コスト関数
都市 $i$ と都市 $j$ の距離を $d_{ij}$、都市数を $N$ として、
$$
 \sum_{n=0}^{N-1} \sum_{i=0}^{N-1} \sum_{j=0}^{N-1} d_{ij} q_{n,i} q_{n+1,j}
$$

In [ ]:
from amplify import sum_poly

cost = sum_poly(
    ncity,
    lambda n: sum_poly(
        ncity,
        lambda i: sum_poly(
            ncity, lambda j: distances[i, j] * q[n, i] * q[(n + 1) % ncity, j]
        ),
    ),
)

### 行の one-hot

In [ ]:
from amplify.constraint import one_hot

# 行に対する制約
row_constraints = [one_hot(q[n]) for n in range(ncity)]

### 列の one-hot

In [ ]:
# 列に対する制約
col_constraints = [one_hot(q[:, i]) for i in range(ncity)]

### <span style="color: red; ">反転対称性の除去</span>
[ 0 7 1 5 6 3 4 2] と [0 2 4 3 6 5 1 7 ] は反転しているが同一の解。
これを除去する制約を追加する。

$ q_{N-1, i} q_{1,j} = 0 \qquad (i<j)$

In [ ]:
from amplify.constraint import penalty

# 順序に対する制約
pem_constraint = [
    penalty(q[ncity - 1, i] * q[1, j])
    for i in range(ncity)
    for j in range(i + 1, ncity)
]

# 下の制約式変更すること

### 制約

In [ ]:
from amplify import sum_poly

constraints = sum(row_constraints) + sum(col_constraints)
# constraints = sum(row_constraints) + sum(col_constraints) + sum(pem_constraint)

constraints *= np.amax(distances)  # 制約条件の強さを設定

### イジングモデル生成

In [ ]:
#######################################################
# 確認用
model = cost + constraints

# model.logical_poly
# model.input_constraints
# model.input_constraints[0][0].penalty

### ソルバ生成

In [ ]:
from amplify import Solver
from amplify.client import FixstarsClient

client = FixstarsClient()
client.token = "rSnIw4H95J8GuAKbvPxzG2PMOCCcFoZD"  #2022-11-27まで有効
client.parameters.timeout = 1000  # タイムアウト1秒

solver = Solver(client)

### ソルバ実行

In [ ]:
result = solver.solve(model)
if len(result) == 0:
    raise RuntimeError("Any one of constraints is not satisfied.")

energy = result[0].energy
values = result[0].values
print('energy = ',energy)

q_values = q.decode(values)
# print('q_values = ',q_values)

print('Execution time =',solver.execution_time, '[msec]')

### ソルバ実行（ループ）

In [ ]:
import csv

# 繰り返し回数
lcount = 10

with open('/Users/stomo/Desktop/temp_qa.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['count', 'energy', 'time'])

    for k in range (lcount):
        result = solver.solve(model)
        if len(result) == 0:
            raise RuntimeError("Any one of constraints is not satisfied.")
        writer.writerow([k, result[0].energy, solver.execution_time])

f.close()

values = result[0].values
q_values = q.decode(values)

### 経路のプロット

In [ ]:
def show_route(route: list, distances: np.ndarray, locations: np.ndarray):

    ncity = len(route)
    path_length = sum(
        [distances[route[i]][route[(i + 1) % ncity]] for i in range(ncity)]
    )

    x = [i[0] for i in locations]
    y = [i[1] for i in locations]
    plt.figure(figsize=(7, 7))
    plt.title(f"path length: {path_length}")
    plt.xlabel("x")
    plt.ylabel("y")

    for i in range(ncity):
        r = route[i]
        n = route[(i + 1) % ncity]
        plt.plot([x[r], x[n]], [y[r], y[n]], "b-")
    plt.plot(x, y, "ro")
    plt.show()

    # return path_length

In [ ]:
route = np.where(np.array(q_values) == 1)[1]
show_route(route, distances, locations)

# <span style="color: blue; ">0-1 整数線形最適化による定式化</span>
## 変数、問題生成

In [ ]:
import networkx as nx
from pulp import *
from itertools import combinations

G = nx.complete_graph(ncity)  # 完全グラフの作成
G = nx.to_directed(G)     # 有向グラフ

nodes = list(G.nodes)     # 頂点集合
edges = list(G.edges)     # 辺集合

# 決定変数
xvars = LpVariable.dicts('x', edges, None, None, LpBinary)
# 巡回順変数
wvars = LpVariable.dicts('w', nodes, 1, ncity-1, LpContinuous)

# 問題生成
TSP = LpProblem('TSP', LpMinimize)

## コスト関数
$d_{i,j}$ : 辺 $\{ i,j \}$ のコスト（距離）
$$
 \sum_{i,j \in V} d_{i,j} x_{i,j}
$$

In [ ]:
TSP += lpSum([distances[e[0]][e[1]] * xvars[e] for e in edges]), 'Total Cost'

## 1-hot 制約
### 流入辺は1本
$$
 \sum_{j \in V, j \neq i} x_{i,j} = 1 \qquad (i \in V)
$$
### 流出辺は1本
$$
 \sum_{i \in V, i \neq j} x_{i,j} = 1 \qquad (j \in V)
$$

In [ ]:
for u in nodes:
    # 流出辺
    TSP += lpSum([xvars[e] for e in edges if e[0] == u]) == 1, 'Flow Conservation (out) %s'%u
    # 流入辺
    TSP += lpSum([xvars[e] for e in edges if e[1] == u]) == 1, 'flow Conservation (in) %s'%u

## 部分順回路除去制約
### MTZ
$w_i$ : 頂点 $i$ の訪問順（ただし、$w_0 = 0$）（頂点 0 は出発点）
$$
 w_i + 1 - (n-1) (1-x_{i,j}) \leq w_j \qquad (i,j = 1, \ldots, n-1, i \neq j)
$$

### 訪問順の上下限
$$
 w_i \geq 2 - x_{0,i} + (n-3) x_{i,0}, \; w_i \leq n-2 + x_{i,0} - (n-3) x_{0,i}
$$

In [ ]:
# 制約: 部分順回路除去　(MTZ)
nodeset = set(nodes)
for (u,v) in [(u,v) for u in nodes for v in nodes if u!= 0 and v!=0 and u!=v]:
    TSP += wvars[u] + 1 - (ncity-1)*(1-xvars[(u,v)]) <= wvars[v]
    TSP += 2 - xvars[(0,u)] + (ncity - 3)*xvars[(u,0)] <= wvars[u]
    TSP += ncity - 2 + xvars[(u,0)] - (ncity - 3)*xvars[(0,u)] >= wvars[u]

### 求解

In [ ]:
# ソルバスレッド数設定
solver = COIN_CMD(threads=10)
# 求解
status = TSP.solve(solver)

print('Status:', LpStatus[status])
print('Optimal val:', value(TSP.objective))
for e in edges:
    if xvars[e].value() > 0.5:
        print(e,xvars[e].value(), ' cost:', distances[e[0]][e[1]])
print('n:', ncity, ', number of constraints:', len(TSP.constraints))

### 求解（ループ）

In [ ]:
import csv
import time

# 繰り返し回数
lcount = 10

# ソルバスレッド数設定
solver = COIN_CMD(threads=10)

with open('/Users/stomo/Desktop/temp_lp.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['count', 'energy', 'time'])

    for k in range (lcount):
        st = time.time()
        status = TSP.solve(solver)
        et = time.time() -st
        if LpStatus[status] != 'Optimal':
            raise RuntimeError("Any one of constraints is not satisfied.")
        writer.writerow([k, value(TSP.objective), et*1000])

f.close()

## 経路のプロット

In [ ]:
import matplotlib.pyplot as plt

path_length = sum([distances[e[0]][e[1]] for e in edges if xvars[e].value() > 0.5])

plt.figure(figsize=(7, 7))
plt.title(f'path length:{path_length}')
plt.xlabel("x")
plt.ylabel("y")

for e in edges:
    if xvars[e].value() > 0.5:
        plt.plot([locations[e[0]][[0]], locations[e[1]][[0]]], [locations[e[0]][[1]], locations[e[1]][[1]]], 'b-')

plt.plot([i[0] for i in locations],[i[1] for i in locations],'ro')
plt.show()